In [1]:
from platform import python_version
python_version()

'3.9.16'

In [4]:
!git clone https://github.com/deep-diver/gpt2-ft-pipeline.git

Cloning into 'gpt2-ft-pipeline'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 5), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [8]:
TARGET_ROOT_DIR = "alpaca"
TARGET_TRAIN_DIR = TARGET_ROOT_DIR + "/train"
TARGET_TEST_DIR = TARGET_ROOT_DIR + "/test"

!mkdir -p {TARGET_TRAIN_DIR}
!mkdir -p {TARGET_TEST_DIR}

In [5]:
import tensorflow_datasets as tfds

builder = tfds.builder("alpaca")
builder.download_and_prepare()

In [6]:
!ls {builder.data_dir}

alpaca-train.tfrecord-00000-of-00001  dataset_info.json
alpaca-val.tfrecord-00000-of-00001    features.json


In [9]:
!cp {builder.data_dir}/alpaca-train.tfrecord-00000-of-00001 {TARGET_TRAIN_DIR}/alpaca-train.tfrecord
!cp {builder.data_dir}/alpaca-val.tfrecord-00000-of-00001 {TARGET_TEST_DIR}/alpaca-val.tfrecord

In [10]:
import tfx
tfx.__version__

'1.13.0'

In [11]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import ImportExampleGen
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Evaluator
from tfx.proto import example_gen_pb2

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental.latest_blessed_model_resolver import LatestBlessedModelResolver

import tensorflow_model_analysis as tfma

2023-06-19 01:32:16.603024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [214]:
context = InteractiveContext()

In [215]:
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train/*.tfrecord"),
        example_gen_pb2.Input.Split(name="eval", pattern="test/*.tfrecord"),
    ]
)
example_gen = ImportExampleGen(
    input_base=TARGET_ROOT_DIR, 
    input_config=input_config
)

In [216]:
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [217]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [218]:
schema_file = "schema.pbtxt"

In [219]:
%%writefile {schema_file}

feature {
  name: "instruction"
  type: BYTES
}

feature {
  name: "input"
  type: BYTES
}

feature {
  name: "output"
  type: BYTES
}

Overwriting schema.pbtxt


In [233]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])

context.run(schema_gen, enable_cache=False)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 6
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [235]:
!cat /var/tmp/tfx-interactive-2023-06-19T02_56_52.346526-tzb47pnx/SchemaGen/schema/6/schema.pbtxt

feature {
  name: "input"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "instruction"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "output"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}


In [236]:
!mkdir modules

mkdir: cannot create directory ‘modules’: File exists


In [237]:
proprocessing_file = "modules/preprocessing.py"
preprocessing_fn = "modules.preprocessing.preprocessing_fn"

In [268]:
%%writefile {proprocessing_file}

import tensorflow as tf

def get_prompt(x):
    def get_prompt_with_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Input:\n"])
        result = tf.strings.join([result, x['input']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        return result

    def get_prompt_without_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        return result

    result = tf.cond(
        tf.math.equal(x['input'], ''),
        get_prompt_with_input,
        get_prompt_without_input
    )

    return {
        'instruction': x['instruction'],
        'input': x['input'],
        'output': result
    }

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    # print(inputs)
    # inputs['combine'] = tf.strings.as_string([3, 2])
    # inputs = tf.map_fn(lambda x: get_prompt(x), inputs)
    # inputs['instruction'] = tf.cast(inputs['instruction'], dtype=tf.string)
    # print(inputs['instruction'])
    inputs = tf.map_fn(lambda x: get_prompt(x), inputs)
    
    return {
        'combine': inputs['output']
    }

Overwriting modules/preprocessing.py


In [269]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn=preprocessing_fn)

In [270]:
context.run(transform, enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 17
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [189]:
model_file = "modules/model.py"
model_fn = "modules.model.run_fn"

In [284]:
%%writefile {model_file}

from typing import List, Dict, Tuple

import absl
import tensorflow as tf
import keras_nlp
from absl import logging
from tensorflow import keras
from tfx.components.trainer.fn_args_utils import FnArgs

from tfx_bsl.tfxio import dataset_options
from tfx.components.trainer.fn_args_utils import DataAccessor

from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import tensorflow_transform as tft

_TRAIN_BATCH_SIZE = 4
_NUM_EPOCHS = 1

_TRAIN_DATA_NUM = 46104

_FEATURE_KEYS = [
    "instruction",
    "input",
    "output",
]

_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.string)
        for feature in _FEATURE_KEYS
    },
}

def INFO(text: str):
    absl.logging.info(text)

signature_dict = {
    "prompt": tf.TensorSpec(shape=[], dtype=tf.string, name="prompt"),
    "max_length": tf.TensorSpec(shape=[], dtype=tf.int64, name="max_length"),
}
    
def get_prompt(x):
    def get_prompt_with_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Input:\n"])
        result = tf.strings.join([result, x['input']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        return result

    def get_prompt_without_input():
        result = tf.strings.join(["### Instruction:\n", x['instruction']])
        result = tf.strings.join([result, '\n\n'])

        result = tf.strings.join([result, "### Response:\n"])
        result = tf.strings.join([result, x['output']])
        return result

    result = tf.cond(
        tf.math.equal(x['input'], ''),
        get_prompt_with_input,
        get_prompt_without_input
    )

    return result    
    
def _input_fn(
    file_pattern: List[str],
    data_accessor: DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    is_train: bool = False,
    batch_size: int = 32,
) -> tf.data.Dataset:
    INFO(f"Reading data from: {file_pattern}")

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size
        ),
        tf_transform_output.transformed_metadata.schema,
    ).map(
        lambda x: x['combine']
    ).prefetch(
        tf.data.AUTOTUNE
    )

    return dataset
    
def _get_gpt2_model(cardinality):
    gpt2_tokenizer = keras_nlp.models.GPT2Tokenizer.from_preset("gpt2_base_en")
    gpt2_preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
        "gpt2_base_en",
        sequence_length=256,
        add_end_token=True,
    )
    gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
        "gpt2_base_en", preprocessor=gpt2_preprocessor
    )
    
    learning_rate = keras.optimizers.schedules.PolynomialDecay(
        5e-5,
        decay_steps=cardinality,
        end_learning_rate=0.0,
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
    )    
    
    return gpt2_lm
    
def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=True,
        batch_size=_TRAIN_BATCH_SIZE,
    )

    model = _get_gpt2_model(train_dataset.cardinality() * _NUM_EPOCHS)
    model.fit(
        train_dataset, 
        steps_per_epoch=_TRAIN_DATA_NUM // _TRAIN_BATCH_SIZE,
        epochs=_NUM_EPOCHS
    )
    
    tf.saved_model.save(
        model,
        fn_args.serving_model_dir,
        signatures={"serving_default": gpt2_lm_exporter(model)},
    )    


Overwriting modules/model.py


In [285]:
trainer = Trainer(
    run_fn=model_fn,
    transformed_examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
)

In [286]:
context.run(trainer, enable_cache=False)

Cause: could not parse the source code of <function _input_fn.<locals>.<lambda> at 0x7fc144564f70>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code of <function _input_fn.<locals>.<lambda> at 0x7fc144564f70>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-06-19 03:19:40.583261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_41' with dtype resource
	 [[{{node Placeholder/_41}}]]
2023-06-19 03:19:40.584473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_37' with dtype resource
	 [[{{node Placeholder/_37}}]]
2023-06-19 03:20:11.141206: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


  126/11526 [..............................] - ETA: 26:29:34 - loss: 1.1375 - accuracy: 0.4047

KeyboardInterrupt: 